Interesting paper https://reader.elsevier.com/reader/sd/pii/S1877050917316599?token=63147E6FB36A421DB648052F0A734C07182BF929F1E6FE4F62F2F0CA6A280FAE311F85835C9CC3F188C17B1458D1A80D

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import IPython.display as ipd # to display audio inside jupyter
#import librosa # Audio parsing
#import librosa.display
import matplotlib.pyplot as plt # to make graphs
import sklearn # Ml
from tqdm import tqdm_notebook as tqdm # progress bar
import multiprocessing # going faster
from multiprocessing import Pool
import time
import glob
import gc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/surgical-mask-create-images/train/fold_spectogram/images'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
images_input =  "surgical-mask-create-images"

In [3]:
# reading competition data
def f_path(fileName, tpe = None):
    """
    Adds full path to a filename recived
    tpe = None / validation / train / test
    """
    if tpe == None:
        return "/kaggle/input/ml-fmi-23-2020/" + fileName
    else:
        return "/kaggle/input/ml-fmi-23-2020/" + tpe + "/" + tpe + "/" + fileName
    
def split_into_files(file):
    """
    Splits the recived input into files
    """
    file = file.split("\n")
    file = [x.split(",") for x in file]
    return file
# Reading the file names
with open(f_path("train.txt"), "r") as f:
    trainFileNames = split_into_files(f.read())
with open(f_path("validation.txt"), "r") as f:
    validationFileNames = split_into_files(f.read())
with open(f_path("test.txt"), "r") as f:
    testFileNames = split_into_files(f.read())

In [4]:
# Reading competition data
trainFileNames = trainFileNames[:-1]
trainOnlyFilesNames = [x[0] for x in trainFileNames]
trainLabels = [int(x[1]) for x in trainFileNames]

validationFileNames = validationFileNames[:-1]
validationOnlyFilesNames = [x[0] for x in validationFileNames]
validationLabels = [int(x[1]) for x in validationFileNames]

testFileNames = testFileNames[:-1]
testOnlyFilesNames = [x[0] for x in validationFileNames]


In [5]:
def read_images_from_dir(directory):
    files = glob.glob(directory + "/*")
    #images = [load_img(x) for x in files]
    images = [load_img(x, grayscale=True) for x in files]
    filesId = [x.split('/')[-1].split('.')[0] for x in files]
    
    return images, filesId

def read_all_images_for(dataType, spectogramType):
    """
    spectogramTypes = fold_spectogram / mfcc_spectogram / crf_spectogram
    
    """
    spectogramDir = "/kaggle/input/"+images_input+"/"+dataType+"/"+spectogramType+"/images"
    spectogram = read_images_from_dir(spectogramDir) 
    return spectogram
    
# Reading image data

trainDir = "/kaggle/input/"+images_input+"/train"
validationDir = "/kaggle/input/"+images_input+"/validation"
testDir = "/kaggle/input/"+images_input+"/test"

test_fold_spectogram = read_all_images_for("test", "fold_spectogram")
# test_mfcc_spectogram = read_all_images_for("test", "mfcc_spectogram")
# test_crf_spectogram = read_all_images_for("test", "crf_spectogram")

train_fold_spectogram = read_all_images_for("train", "fold_spectogram")
# train_mfcc_spectogram = read_all_images_for("train", "mfcc_spectogram")
# train_crf_spectogram = read_all_images_for("train", "crf_spectogram")

validation_fold_spectogram = read_all_images_for("validation", "fold_spectogram")
# validation_mfcc_spectogram = read_all_images_for("validation", "mfcc_spectogram")
# validation_crf_spectogram = read_all_images_for("validation", "crf_spectogram")

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [6]:
# # Combining images
# validation_spectogram = [[],[]]

# for i in range(len(validation_fold_spectogram[0])):
#     validation_spectogram[0].append((validation_fold_spectogram[0][i], validation_mfcc_spectogram[0][i], validation_crf_spectogram[0][i]))
# validation_spectogram[1] = validation_fold_spectogram[1]

# train_spectogram = [[],[]]

# for i in range(len(train_fold_spectogram[0])):
#     train_spectogram[0].append((train_fold_spectogram[0][i], train_mfcc_spectogram[0][i], train_crf_spectogram[0][i]))
#     train_spectogram[1] = train_fold_spectogram[1]

# test_spectogram = [[],[]]

# for i in range(len(test_fold_spectogram[0])):
#     test_spectogram[0].append((test_fold_spectogram[0][i], test_mfcc_spectogram[0][i], test_crf_spectogram[0][i]))
#     test_spectogram[1] = test_fold_spectogram[1]

In [7]:
# def augment_data_horizontal(spectogram):
#     images= spectogram[0]
#     names = spectogram[1]
#     augmented_data = []
#     augmented_names = []
#     for i in range(len(images)):
#         image = images[i]
#         name = names[i]
#         image_ar = img_to_array(image)
#         image_ar = np.flip(image_ar, axis = 1)
#         augmented_image = array_to_img(image_ar)
        
#         augmented_data.append(image)
#         augmented_data.append(augmented_image)
#         augmented_names.append(name)
#         augmented_names.append(name)

#     return augmented_data, augmented_names

# def augment_data_vertical(spectogram):
#     images= spectogram[0]
#     names = spectogram[1]
#     augmented_data = []
#     augmented_names = []
#     for i in range(len(images)):
#         image = images[i]
#         name = names[i]
#         image_ar = img_to_array(image)
#         image_ar = np.flip(image_ar, axis = 0)
#         augmented_image = array_to_img(image_ar)
        
#         augmented_data.append(image)
#         augmented_data.append(augmented_image)
#         augmented_names.append(name)
#         augmented_names.append(name)
#     return augmented_data, augmented_names 

In [8]:
#augmented_train_fold_spectogram = augment_data_horizontal(train_fold_spectogram)
#augmented_train_fold_spectogram = augment_data_vertical(augmented_train_fold_spectogram)

In [9]:
class DataGenerator(keras.utils.Sequence):
    def __image_processing(self, image):
        image_ar = img_to_array(image)
        return image_ar
        #return np.concatenate((img_to_array(image[0]), img_to_array(image[1]), img_to_array(image[2])), axis = 2) # for 3 images in one
    def __get_label_from_aid(self,aid):
        try:
            index = trainOnlyFilesNames.index(str(aid) + ".wav")
            return trainLabels[index]
        except:
            pass
        try:
            index = validationOnlyFilesNames.index(str(aid) + ".wav")
            return validationLabels[index]
        except:
            pass
        return -1
    def __split_image_list_into_labels(self, list_images):
        images = []
        for i in range(len(list_images[0])):
            images.append(
                (list_images[0][i], list_images[1][i]))   
        return images
    
    def __init__(self, list_images, batch_size=32, dim=(32, 32, 32), n_channels=1,
                 n_classes=2, shuffle=True, augment=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_images = self.__split_image_list_into_labels(list_images)
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_images_temp):
        # X : (n_samples, *dim, n_channels)
        'Generates data containing batch_size samples'
        # Initialization
        #X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)
        X = [0] * self.batch_size
        y = [0] * self.batch_size
        # Generate data
        for i, images in enumerate(list_images_temp):
            # Store sample
            X[i] = self.__image_processing(images[0])
            # Store class
            y[i] = self.__get_label_from_aid(images[1])
        
        X = np.asarray(X)
        y = np.asarray(y)
        return X, y

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of images
        list_images_temp = [self.list_images[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_images_temp)

        return X, y


In [10]:
def get_label_from_aid(aid):
    try:
        index = trainOnlyFilesNames.index(str(aid) + ".wav")
        return trainLabels[index]
    except:
        pass
    try:
        index = validationOnlyFilesNames.index(str(aid) + ".wav")
        return validationLabels[index]
    except:
        pass
    return -1

def preproces_images(images):
    newImages = [img_to_array(x) for x in images]
    return newImages

def generate_data_and_labels(data):
    images, fileNames = data
    
    X = preproces_images(images)
    Y = [get_label_from_aid(x) for x in fileNames]
    
    return X, Y

def generate_labels(data):
    images, fileNames = data
    
    Y = [get_label_from_aid(x) for x in fileNames]
    
    return Y

In [11]:
# X_train, Y_train = generate_data_and_labels(train_fold_spectogram)
# X_validation, Y_validation = generate_data_and_labels(validation_fold_spectogram)

# X_train = np.asarray(X_train)
# X_validation = np.asarray(X_validation)

# Y_train = np.asarray(Y_train)
# Y_validation = np.asarray(Y_validation)
# X_test = np.array(generate_data_and_labels(test_fold_spectogram)[0])

# del train_fold_spectogram
# del validation_fold_spectogram
# gc.collect()

In [12]:
X_validation, Y_validation = generate_data_and_labels(validation_fold_spectogram)
X_test = np.array(generate_data_and_labels(test_fold_spectogram)[0])
X_validation = np.asarray(X_validation)

In [13]:
Y_train = generate_labels(train_fold_spectogram)
Y_train = np.asarray(Y_train)

In [14]:
train_generator = DataGenerator(train_fold_spectogram)
validation_generator = DataGenerator(validation_fold_spectogram)

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, LSTM, Bidirectional, GlobalMaxPooling1D, Conv2D, Dropout, MaxPooling2D,Input,ThresholdedReLU, Flatten, BatchNormalization, Activation, GlobalMaxPooling2D, concatenate, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, Callback, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, Callback, EarlyStopping
import sklearn
from tensorflow.keras.applications.resnet50 import ResNet50



In [31]:
# main_input = Input(shape = (217, 223, 1))
# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
# x = BatchNormalization()(x)
# x = MaxPooling2D((3,3))(x)
# skip1 = x
# x = Activation("relu")(x)

# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# skip2 = concatenate([skip1, x])
# x = skip2
# x = Activation("relu")(x)

# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# skip3 = concatenate([skip2, x])
# x = skip3
# x = Activation("relu")(x)

# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# skip4 = concatenate([skip3, x])
# x = skip4
# x = Activation("relu")(x)

# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
# x = BatchNormalization()(x)
# skip5 = concatenate([skip4, x])
# x = skip5
# x = Activation("relu")(x)
# x = MaxPooling2D((2,2))(x)
# x = Flatten()(x)


# x = Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
# x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
# output = Dense(1, activation='sigmoid')(x)

# model = Model(inputs=main_input, outputs=output)
# model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
main_input = Input(shape = (217, 223, 1))
x = Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D((3,3))(x)

x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D((3,3))(x)

x = Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D((3,3))(x)
x = Flatten()(x)

x = Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=main_input, outputs=output)
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
#model.fit(X_train, Y_train, epochs = 50, validation_data= (X_validation, Y_validation),  batch_size = 32)
class_weight = sklearn.utils.class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
mcp_save = ModelCheckpoint('model.hdf5', save_best_only=True, monitor='val_loss', mode='min')

model.fit(x = train_generator, epochs = 200, validation_data=validation_generator, shuffle=True,
                         class_weight = class_weight, 
                callbacks= [
                              EarlyStopping(patience=10, monitor='val_loss', mode='min'),
                              mcp_save,
                              ReduceLROnPlateau(factor=.3)
                         ])

Train for 250 steps, validate for 31 steps
Epoch 1/200
250/250 [==============================] - 9s 37ms/step - loss: 1.4003 - accuracy: 0.5469 - val_loss: 0.7967 - val_accuracy: 0.5917
Epoch 2/200
250/250 [==============================] - 8s 33ms/step - loss: 0.8276 - accuracy: 0.6059 - val_loss: 0.6396 - val_accuracy: 0.6411
Epoch 3/200
250/250 [==============================] - 8s 32ms/step - loss: 0.6538 - accuracy: 0.6484 - val_loss: 0.8837 - val_accuracy: 0.5393
Epoch 4/200
250/250 [==============================] - 8s 33ms/step - loss: 0.6187 - accuracy: 0.6747 - val_loss: 0.6137 - val_accuracy: 0.6683
Epoch 5/200
250/250 [==============================] - 8s 33ms/step - loss: 0.5553 - accuracy: 0.7210 - val_loss: 0.6109 - val_accuracy: 0.6734
Epoch 6/200
250/250 [==============================] - 8s 34ms/step - loss: 0.5184 - accuracy: 0.7383 - val_loss: 0.6768 - val_accuracy: 0.6502
Epoch 7/200
250/250 [==============================] - 8s 33ms/step - loss: 0.4801 - accuracy

In [43]:
model.load_weights('model.hdf5')
predictions = model.predict(X_test).round()
finalPredictions = predictions

In [44]:
validation_predictions = model.predict(X_validation).round()
from sklearn.metrics import accuracy_score
accuracy_score(Y_validation, validation_predictions)

0.716

In [ ]:
# full_data = np.concatenate((train_fold_spectogram, validation_fold_spectogram))
# Y_full = np.concatenate((Y_train, Y_validation))

In [ ]:
# def create_model():
#     main_input = Input(shape = (217, 223, 3))
#     x = Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = MaxPooling2D((2,2))(x)

#     x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = MaxPooling2D((2,2))(x)

#     x = Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same')(main_input)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     x = MaxPooling2D((2,2))(x)
#     x = Flatten()(x)

#     x = Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
#     x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
#     output = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs=main_input, outputs=output)
#     model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# from sklearn.model_selection import KFold



# curentFold = 0
# kf = KFold(n_splits=10, shuffle = True, random_state = 42)
# for train_index, test_index in kf.split(X_full):
    
#     curentFold += 1
#     gc.collect()
#     X_train_fold = np.array([X_full[x] for x in train_index])
#     X_test_fold = np.array([X_full[x] for x in test_index])
    
#     Y_train_fold = np.array([Y_full[x] for x in train_index])
#     Y_test_fold = np.array([Y_full[x] for x in test_index])
    
#     train_generator = DataGenerator(train_fold_spectogram)
#     validation_generator = DataGenerator(validation_fold_spectogram)
    
#     class_weight = sklearn.utils.class_weight.compute_class_weight('balanced',np.unique(Y_train_fold), Y_train_fold)
    
#     model=create_model()
#     mcp_save = ModelCheckpoint('modelfold'+str(curentFold)+'.hdf5', save_best_only=True, monitor='val_loss', mode='min')
#     model.fit(x = train_generator, epochs = 200, validation_data=validation_generator, shuffle=True,
#                          class_weight = class_weight, 
#                 callbacks= [
#                               EarlyStopping(patience=10, monitor='val_loss', mode='min'),
#                               mcp_save,
#                               ReduceLROnPlateau(factor=.3)
#                          ])
    
    
#     print("Fold "+str(curentFold)+" done.")
    

In [ ]:
# validation_fold_spectogram = read_all_images_for("validation", "fold_spectogram")
# X_validation, Y_validation = generate_data_and_labels(validation_fold_spectogram)
# X_validation = np.array(X_validation)

In [ ]:
# # 6  didn't learn anything
# predictions = []
# model=create_model()
# model.load_weights('modelfold1.hdf5')

# for i in range(1,11):
#     if i == 8 or i == 4:
#         continue
#     model.load_weights('modelfold'+str(i)+'.hdf5')
#     predictions.append(model.predict(X_validation))
# finalPredictions = np.average(predictions, axis = 0)
# finalPredictions = finalPredictions.round()
# from sklearn.metrics import accuracy_score
# accuracy_score(Y_validation, finalPredictions)

In [ ]:
# predictions = []
# model=create_model()
# model.load_weights('modelfold1.hdf5')

# for i in range(1,11):
#     if i == 8 or i == 4:
#         continue
#     model.load_weights('modelfold'+str(i)+'.hdf5')
#     predictions.append(model.predict(X_test))
# finalPredictions = np.average(predictions, axis = 0)
# finalPredictions = finalPredictions.round()


In [45]:
test_filenames = test_fold_spectogram[1]


In [46]:
with open("submisie5.txt", "w+") as f:
    f.write("name,label\n")
    for i in range(0, len(test_filenames)):
        f.write(test_filenames[i] + ".wav," + str(int(finalPredictions[i][0]))+"\n")